## Цель ноутбука

Построить и обучить свёрточную нейронную сеть для задачи классификации на датасете [FashionMNIST](https://pytorch.org/vision/stable/generated/torchvision.datasets.FashionMNIST.html).


### 1. Устанавливаем и импортируем необходимые библиотеки.

In [1]:
!pip install numpy
!pip install matplotlib
!pip install torch torchvision torchinfo

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Compose, Normalize
from torchinfo import summary
from torchvision import datasets
import matplotlib.pyplot as plt

### 2. Готовим данные.

In [3]:
dataset = datasets.FashionMNIST('data/', download=True)

In [5]:
mapping = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

In [ ]:
fig, axs = plt.subplots(1, 10, figsize=(20, 5))
for i in range(10):
    axs[i].imshow(dataset[i][0], cmap='gray')
    axs[i].set_title(mapping[dataset[i][1]])
    axs[i].axis('off')
plt.show()

In [7]:
transform = Compose([
    ToTensor(),
    Normalize([0.5], [0.5])
])

train_dataset = datasets.FashionMNIST('data/', train=True, download=True, transform=transform)
val_dataset = datasets.FashionMNIST('data/', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1000)

### 3. Собираем модель.
- Свёрточным слоем получаем признаки.
- Сокращаем их количество с помощью MaxPooling.
- Строим перцептрон на полученных признаках.

In [9]:
class CNN(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=1,
                      out_channels=32,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            nn.Flatten(),

            nn.Linear(in_features=6272, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=n_classes)
        )

    def forward(self,x):
        print(x.size())
        return self.model(x)

In [11]:
net = CNN(10)
summary(net, input_size=(1, 1, 28, 28))

torch.Size([1, 1, 28, 28])


Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [1, 10]                   --
├─Sequential: 1-1                        [1, 10]                   --
│    └─Conv2d: 2-1                       [1, 32, 28, 28]           320
│    └─ReLU: 2-2                         [1, 32, 28, 28]           --
│    └─MaxPool2d: 2-3                    [1, 32, 14, 14]           --
│    └─Flatten: 2-4                      [1, 6272]                 --
│    └─Linear: 2-5                       [1, 64]                   401,472
│    └─ReLU: 2-6                         [1, 64]                   --
│    └─Linear: 2-7                       [1, 10]                   650
Total params: 402,442
Trainable params: 402,442
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.65
Input size (MB): 0.00
Forward/backward pass size (MB): 0.20
Params size (MB): 1.61
Estimated Total Size (MB): 1.81

### 4. Обучаем модель и сохраняем веса.
Пишем функции `train()` и `validate()`. Создаем лосс и оптимизатор, фиксируем гиперпараметры.

In [13]:
def train(model, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre):
    model.train()
    for epoch in range(n_epoch):
        loss_sum = 0
        print(f'Epoch: {epoch}')
        for step, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data).squeeze(1)
            loss = loss_f(output, target)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()

            if step % 10 == 0:
                print(f'Iter: {step} \tLoss: {loss.item()}')

        print(f'Mean Train Loss: {loss_sum / (step + 1):.6f}', end='\n\n')

        if epoch % val_fre == 0:
            validate(model, val_loader)

def validate(model, val_loader):
    model.eval()
    loss_sum = 0
    correct = 0
    for step, (data, target) in enumerate(val_loader):
        with torch.no_grad():
            output = model(data).squeeze(1)
            loss = loss_f(output, target)
        loss_sum += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    acc = correct / len(val_loader.dataset)
    print(f'Val Loss: {loss_sum / (step + 1):.6f} \tAccuracy: {acc}')
    model.train()

In [17]:
model = CNN(10)
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

n_epoch = 50
val_fre = 2

train(model, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre)
# torch.save(model.state_dict(), 'checkpoints/cnn.pth')
# model.load_state_dict(torch.load('checkpoints/cnn.pth'))
validate(model, val_loader)

Epoch: 0
torch.Size([1000, 1, 28, 28])
Iter: 0 	Loss: 2.298125982284546
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
Iter: 10 	Loss: 1.6210345029830933
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
Iter: 20 	Loss: 1.2804856300354004
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 1, 28, 28])
torch.Size([1000, 

KeyboardInterrupt: 